# Importing packages

In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import regex as re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import requests

C:\Users\vella\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vella\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vella\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importing data

In [5]:
df = pd.read_csv(r"C:\Users\vella\Desktop\Praju\College\Projects\Final year\Data\Tesla\stock_tweets.csv")
df.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [14]:
df.rename(columns = {'Stock Name':'Stock_Name'}, inplace = True)

In [15]:
names = ['TSLA']
df = df[df.Stock_Name.isin(names)]

In [16]:
df

,Date,Tweet,Stock_Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."
...,...,...,...,...
37417,2021-09-30 02:52:38+00:00,Playing in the dirt and #chasingsunsets\n@tesl...,TSLA,"Tesla, Inc."
37418,2021-09-30 02:40:26+00:00,I agree with @freshjiva that $TSLA ‘s EV busin...,TSLA,"Tesla, Inc."
37419,2021-09-30 01:59:02+00:00,Hold. On. Tight. $TSLA,TSLA,"Tesla, Inc."
37420,2021-09-30 01:38:26+00:00,Get ready for a $TSLA _ _ _ _ _ _ Q3 delivery...,TSLA,"Tesla, Inc."


In [17]:
df1 = df.iloc[:,[0,1]]
df1.head()

,Date,Tweet
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,..."


In [18]:
df1['Date'] = df1['Date'].str[:10]
df1

C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Tweet
0,2022-09-29,Mainstream media has done an amazing job at br...
1,2022-09-29,Tesla delivery estimates are at around 364k fr...
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,..."
...,...,...
37417,2021-09-30,Playing in the dirt and #chasingsunsets\n@tesl...
37418,2021-09-30,I agree with @freshjiva that $TSLA ‘s EV busin...
37419,2021-09-30,Hold. On. Tight. $TSLA
37420,2021-09-30,Get ready for a $TSLA _ _ _ _ _ _ Q3 delivery...


# Processing the tweets

In [19]:
#Extracting all required values into respective lists
tweets = df1['Tweet']

In [20]:
#STEP 1: Removing mentions and urls from tweet messages
tweets_without_mention = []
for message in tweets:
    tweets_without_mention.append(re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "",message))

In [21]:
#STEP 2: Removing consecutive non-ascii characters
tweets_without_non_ascii = []
for message in tweets_without_mention:
    tweets_without_non_ascii.append(re.sub(r'[^\x00-\x7F]+',' ', message))

In [22]:
#STEP 3: Tokenization and remove punctuations
tweets_tokens = []
for message in tweets_without_non_ascii:
    words = word_tokenize(message)
    words_without_punctuation = [word for word in words if word.isalnum()]
    tweets_tokens.append(words_without_punctuation)

In [23]:
#STEP 4: Removing stop words
stop_words = set(stopwords.words('english'))
tweets_without_stop_words = [] 
  
for i in tweets_tokens:
    temp = []
    for j in i:
        if j not in stop_words: 
            temp.append(j)
    tweets_without_stop_words.append(temp)

In [24]:
#STEP 5: Removing emoticons from tweets
tweets_without_emoticons=[]
for message in tweets_without_stop_words:
    temp=[]
    for word in message:
        to_remove_pattern = re.compile(pattern = "["
                                       u"\U0001F600-\U0001F64F"  # emoticons
                                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                       "]+", flags = re.UNICODE)
        temp.append(to_remove_pattern.sub(r'',word))
    tweets_without_emoticons.append(temp)

In [25]:
#STEP 6: Final processed tweet messages
processed_tweets= []
for message in tweets_without_emoticons:
    s = " "
    s = s.join(message)
    processed_tweets.append(s)

In [26]:
df1['Processed_tweet'] = processed_tweets

C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
df1

,Date,Tweet,Processed_tweet
0,2022-09-29,Mainstream media has done an amazing job at br...,Mainstream media done amazing job brainwashing...
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,Tesla delivery estimates around 364k analysts ...
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,Even I include unvested RSUs additional equity...
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,Hahaha still trying stop Tesla FSD bro Get shi...
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",Stop trying kill kids sad deranged old man
...,...,...,...
37417,2021-09-30,Playing in the dirt and #chasingsunsets\n@tesl...,Playing dirt chasingsunsets
37418,2021-09-30,I agree with @freshjiva that $TSLA ‘s EV busin...,I agree TSLA EV business alone worth gt FSD Wh...
37419,2021-09-30,Hold. On. Tight. $TSLA,Hold On Tight TSLA
37420,2021-09-30,Get ready for a $TSLA _ _ _ _ _ _ Q3 delivery...,Get ready TSLA Q3 delivery number Have ur answer


# Calculating Sentiment Score

In [28]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vella\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [29]:
df1['scores'] = df1['Processed_tweet'].apply(lambda Processed_tweet: sid.polarity_scores(Processed_tweet))
df1.head()

,Date,Tweet,Processed_tweet,scores
0,2022-09-29,Mainstream media has done an amazing job at br...,Mainstream media done amazing job brainwashing...,"{'neg': 0.196, 'neu': 0.627, 'pos': 0.177, 'co..."
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,Tesla delivery estimates around 364k analysts ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,Even I include unvested RSUs additional equity...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp..."
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,Hahaha still trying stop Tesla FSD bro Get shi...,"{'neg': 0.345, 'neu': 0.474, 'pos': 0.181, 'co..."
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",Stop trying kill kids sad deranged old man,"{'neg': 0.667, 'neu': 0.333, 'pos': 0.0, 'comp..."


In [30]:
df1['compound']  = df1['scores'].apply(lambda score_dict: score_dict['compound'])

In [31]:
df1.head(50)

,Date,Tweet,Processed_tweet,scores,compound
0,2022-09-29,Mainstream media has done an amazing job at br...,Mainstream media done amazing job brainwashing...,"{'neg': 0.196, 'neu': 0.627, 'pos': 0.177, 'co...",0.0772
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,Tesla delivery estimates around 364k analysts ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,Even I include unvested RSUs additional equity...,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",0.2960
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,Hahaha still trying stop Tesla FSD bro Get shi...,"{'neg': 0.345, 'neu': 0.474, 'pos': 0.181, 'co...",-0.7096
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",Stop trying kill kids sad deranged old man,"{'neg': 0.667, 'neu': 0.333, 'pos': 0.0, 'comp...",-0.8750
5,2022-09-29,@RealDanODowd @Tesla This is you https://t.co/...,This,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
6,2022-09-29,For years @WholeMarsBlog viciously silenced @T...,For years viciously silenced critics Failing s...,"{'neg': 0.528, 'neu': 0.398, 'pos': 0.074, 'co...",-0.9325
7,2022-09-29,$NIO just because I'm down money doesn't mean ...,NIO I money mean bad investment The whole mark...,"{'neg': 0.24, 'neu': 0.519, 'pos': 0.24, 'comp...",0.2500
8,2022-09-29,50 likes for some $SPY $TSLA charts to study!\...,50 likes SPY TSLA charts study,"{'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'comp...",0.4215
9,2022-09-29,@MrJames__321 @KellyRoofing @TeslaSolar @elonm...,The powerwalls waterproof could bad wiring hou...,"{'neg': 0.478, 'neu': 0.522, 'pos': 0.0, 'comp...",-0.7506


In [32]:
df2 = df1.iloc[:,[0,4]]

In [33]:
df2

,Date,compound
0,2022-09-29,0.0772
1,2022-09-29,0.0000
2,2022-09-29,0.2960
3,2022-09-29,-0.7096
4,2022-09-29,-0.8750
...,...,...
37417,2021-09-30,-0.1531
37418,2021-09-30,0.7003
37419,2021-09-30,0.0000
37420,2021-09-30,0.4215


In [34]:
Tweet_Sent = df2.groupby("Date")["compound"].mean()

In [35]:
Tweet_Sent.head(60)

Date
2021-09-30    0.213078
2021-10-01    0.224785
2021-10-02    0.255568
2021-10-03    0.257956
2021-10-04    0.106842
2021-10-05    0.083851
2021-10-06    0.223397
2021-10-07    0.192353
2021-10-08    0.230448
2021-10-09    0.303168
2021-10-10    0.266588
2021-10-11    0.205428
2021-10-12    0.200554
2021-10-13    0.146121
2021-10-14    0.192803
2021-10-15    0.092672
2021-10-16    0.235861
2021-10-17    0.222597
2021-10-18    0.215244
2021-10-19    0.182304
2021-10-20    0.138617
2021-10-21    0.197119
2021-10-22    0.210288
2021-10-23    0.240073
2021-10-24    0.325569
2021-10-25    0.198978
2021-10-26    0.206996
2021-10-27    0.142435
2021-10-28    0.186397
2021-10-29    0.149483
2021-10-30    0.310728
2021-10-31    0.295694
2021-11-01    0.217334
2021-11-02    0.164199
2021-11-03    0.188551
2021-11-04    0.219168
2021-11-05    0.243579
2021-11-06    0.227581
2021-11-07    0.235930
2021-11-08    0.156180
2021-11-09    0.166692
2021-11-10    0.177834
2021-11-11    0.232207
2021-1

In [36]:
Tweet_Sent = pd.DataFrame(Tweet_Sent)

In [37]:
Tweet_Sent

,compound
Date,
2021-09-30,0.213078
2021-10-01,0.224785
2021-10-02,0.255568
2021-10-03,0.257956
2021-10-04,0.106842
...,...
2022-09-25,0.119892
2022-09-26,0.170785
2022-09-27,0.192374


# Exporting as csv

In [38]:
#Tweet_Sent.to_csv("Tweet_Sent.csv")